In [ ]:
from transformers import pipeline  #mask test

unmasker = pipeline('fill-mask', model='bert-base-multilingual-cased')

unmasker("ชนิดประโยคในภาษา [MASK] มีอะไรบ้าง")


In [ ]:
from transformers import pipeline

classifier = pipeline("text-classification", model="./ecommerce_model", tokenizer="./ecommerce_model")

text = "มีสินค้าในสต๊อกไหม"
result = classifier(text)
print(result)

Device set to use cpu
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


[{'label': 'LABEL_1', 'score': 0.9994086027145386}]


In [ ]:
import pandas as pd

# Sample dataset for e-commerce chat classification
data = {
    "text": [
        "What is the price of iPhone?",
        "สินค้าหมดเมื่อไหร่จะมาอีก?",
        "I want to cancel my order.",
        "ต้องการเปลี่ยนที่อยู่จัดส่ง",
        "How can I track my order?",
        "ได้รับสินค้าผิดชิ้น",
        "The product is damaged.",
        "ขอคืนเงิน",
        "I want to give feedback.",
        "บริการลูกค้าแย่มาก"
    ],
    "label": [
        "Product Inquiry",
        "Product Inquiry",
        "Order Management",
        "Order Management",
        "Order Management",
        "Customer Support",
        "Customer Support",
        "Customer Support",
        "Feedback & Complaints",
        "Feedback & Complaints"
    ]
}

# Create DataFrame
df = pd.DataFrame(data)

# Save to CSV
file_path = "ecommerce_chat_dataset.csv"
df.to_csv(file_path, index=False)

file_path


In [1]:
# !pip install tensorflow-cpu
# !pip install tf-keras
!pip install wandb
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.5 MB/s eta 0:00:00


In [28]:
import wandb
wandb.login()  # Or wandb.login(key="your-api-key") if you want to specify the API key

# Log in to W&B using your API key
#wandb.login(key="local-3be8233374a796b3fed2248de19c52b03220d770")


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [29]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch
from datasets import Dataset
# import wandb
# !wandb login

# Log in to wandb using the API key (if needed)

# Load dataset
df = pd.read_csv("jesus_bless_me.csv")
labels = df['label'].unique()
label2id = {label: i for i, label in enumerate(labels)}
df['label'] = df['label'].map(label2id)

# Split data
train_texts, test_texts, train_labels, test_labels = train_test_split(df['text'], df['label'], test_size=0.2)

# Tokenization
model_name = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch['text'], padding="max_length", truncation=True, max_length=128)

train_dataset = Dataset.from_dict({"text": train_texts.tolist(), "label": train_labels.tolist()}).map(tokenize, batched=True)
test_dataset = Dataset.from_dict({"text": test_texts.tolist(), "label": test_labels.tolist()}).map(tokenize, batched=True)

# Load Model
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(labels))

# Training Setup
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=6,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",  # This evaluates the model at the end of each epoch
    save_strategy="epoch",       # This saves the model at the end of each epoch (matches evaluation)
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    learning_rate=1e-5,  # Adjust this value based on your model and dataset
    weight_decay=0.01  # Helps prevent overfitting
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Train
trainer.train()

# Save Model
model.save_pretrained("./ecommerce_model")
tokenizer.save_pretrained("./ecommerce_model")

# Evaluate
trainer.evaluate()


Map:   0%|          | 0/3234 [00:00<?, ? examples/s]

Map:   0%|          | 0/809 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.002200,0.061518
2,0.000700,0.034995
3,0.000400,0.000359
4,0.000300,0.000253
5,0.000300,0.000209
6,0.000300,0.000195


{'eval_loss': 0.00019493632134981453,
 'eval_runtime': 6.1349,
 'eval_samples_per_second': 131.868,
 'eval_steps_per_second': 16.626,
 'epoch': 6.0}

In [5]:
label2id

{'Product Inquiry': 0, 'Order Management': 1, 'Customer Support': 2}

In [ ]:
import pandas as pd
from transformers import pipeline

# Load dataset
df = pd.read_csv("ecommerce_chat_dataset_thai_2000.csv")
# Initialize the classifier
classifier = pipeline("text-classification", model="./ecommerce_model", tokenizer="./ecommerce_model")
# Loop through the 'text' column
for text in df['text']:
    #print(text)

    # Loop through the 'text' column and classify the text
    df['result'] = df['text'].apply(lambda text: classifier(text)[0]['label'])  # Assuming you want the label

    # Optionally, save the updated DataFrame to a new CSV
    df.to_csv("ecommerce_chat_with_results.csv", index=False)

    # Show the updated DataFrame (optional)
    #print(df.head())



In [32]:
import pandas as pd
from transformers import pipeline

# Load dataset
#df = pd.read_csv("ecommerce_chat_dataset_thai_2000.csv")
# Initialize the classifier
classifier = pipeline("text-classification", model="./ecommerce_model", tokenizer="./ecommerce_model")
# Perform classification
result = classifier("มีrx580อยู่ไหม")

# Extract label
predicted_label = result[0]['label']
score = result[0]['score']
#print(predicted_label)
if predicted_label == 'LABEL_0':
    print("Product Inquiry")
    print(score)
elif predicted_label == 'LABEL_1':
    print("Order Management")
    print(score)
elif predicted_label == 'LABEL_2':
    print("Customer Support")
    print(score)


#{'Product Inquiry': 0, 'Order Management': 1, 'Customer Support': 2}


#print(classifier[0])
# Loop through the 'text' column
# for text in df['text']:
#     #print(text)

#     # Loop through the 'text' column and classify the text
#     df['result'] = df['text'].apply(lambda text: classifier(text)[0]['label'])  # Assuming you want the label

#     # Optionally, save the updated DataFrame to a new CSV
#     df.to_csv("ecommerce_chat_with_results.csv", index=False)

#     # Show the updated DataFrame (optional)
#     #print(df.head())


Device set to use cuda:0


Product Inquiry
0.9992197751998901


In [31]:
!zip -r jesus.zip ecommerce_model/ logs/ results/ wandb/ jesus_bless_me.csv


  adding: ecommerce_model/ (stored 0%)
  adding: ecommerce_model/model.safetensors (deflated 7%)
  adding: ecommerce_model/tokenizer_config.json (deflated 75%)
  adding: ecommerce_model/config.json (deflated 55%)
  adding: ecommerce_model/special_tokens_map.json (deflated 42%)
  adding: ecommerce_model/vocab.txt (deflated 45%)
  adding: logs/ (stored 0%)
  adding: logs/events.out.tfevents.1742142496.f8fc47106181.441.3 (deflated 70%)
  adding: logs/events.out.tfevents.1742143349.f8fc47106181.441.4 (deflated 25%)
  adding: results/ (stored 0%)
  adding: results/checkpoint-405/ (stored 0%)
  adding: results/checkpoint-405/scheduler.pt (deflated 55%)
  adding: results/checkpoint-405/training_args.bin (deflated 52%)
  adding: results/checkpoint-405/model.safetensors (deflated 7%)
  adding: results/checkpoint-405/config.json (deflated 55%)
  adding: results/checkpoint-405/trainer_state.json (deflated 77%)
  adding: results/checkpoint-405/rng_state.pth (deflated 25%)
  adding: results/checkpo